# K-Means Clustering
 
### Authors

| Name | Roll Number |
| - | - |
| Gautam Singh | CS21BTECH11018 |
| Jaswanth Beere | BM21BTECH11007 |

This `.ipynb` file performs K-means clustering on the SVHN dataset, with and
without PCA. It analyses the variations in accuracy and error under different
constraints. 

## Package Imports

The packages required for this exercise are listed below.